In [21]:
import pandas as pd
from collections import defaultdict
from typing import Iterator, Tuple, Dict, List

In [22]:
def length_category(length: int) -> str:
    if length < 100:
        return 'short'
    elif length <= 500:
        return 'medium'
    return 'long'

In [23]:
def do_map_length_helpfulness(row) -> Iterator[Tuple[str, float]]:
    review_text = row.get('review/text', '')
    if not isinstance(review_text, str) or not review_text.strip():
        return
    length = len(review_text.split())
    helpfulness = row.get('review/helpfulness', '0/1')
    try:
        a, b = map(int, str(helpfulness).split('/'))
        ratio = a / b if b > 0 else 0
    except (ValueError, ZeroDivisionError):
        ratio = 0
    yield (length_category(length), ratio)

In [32]:
def do_shuffle_length_helpfulness(mapped_data: Iterator[Tuple[str, float]]) -> Dict[str, List[float]]:
    groups = defaultdict(list)
    for key, value in mapped_data:
        groups[key].append(value)
    return groups

In [25]:
def do_reduce_length_helpfulness(grouped_data: Dict[str, List[float]]) -> List[Tuple[str, float]]:
    results = []
    for cat, values in grouped_data.items():
        avg_ratio = sum(values) / len(values) if values else 0
        results.append((cat, avg_ratio))
    return results

In [36]:
def do_map_genre_length(row) -> Iterator[Tuple[str, int]]:
    review_text = row.get('review/text', '')
    if not isinstance(review_text, str) or not review_text.strip():
        return
    
    length = len(review_text.split())
    
    categories = row.get('categories', '')
    genre = 'Unknown'
    
    if isinstance(categories, str) and categories.strip():
        try:
            import ast
            genre_list = ast.literal_eval(categories)
            if isinstance(genre_list, list) and genre_list:
                genre = genre_list[0]  # Берем первый жанр
            else:
                genre = categories.split(',')[0].strip("['\"]")
        except:
            genre = categories.split(',')[0].strip("['\"]")
    
    yield (genre, length)

In [27]:
def do_shuffle_genre_length(mapped_data: Iterator[Tuple[str, int]]) -> Dict[str, List[int]]:
    groups = defaultdict(list)
    for genre, length in mapped_data:
        groups[genre].append(length)
    return groups

In [28]:
def do_reduce_genre_length(grouped_data: Dict[str, List[int]]) -> List[Tuple[str, float]]:
    results = []
    for genre, lengths in grouped_data.items():
        avg_length = sum(lengths) / len(lengths) if lengths else 0
        results.append((genre, avg_length))
    return results

In [29]:
def demonstrate_map_reduce():
    print("\n=== ДЕМОНСТРАЦИЯ РАБОТЫ MAP-REDUCE ===")
    
    # Пример данных для демонстрации
    sample_data = [
        {'review/text': 'Короткий отзыв', 'review/helpfulness': '3/5'},
        {'review/text': 'Средний по длине отзыв, который содержит больше информации', 'review/helpfulness': '8/10'},
        {'review/text': 'Очень длинный и подробный отзыв, который содержит много деталей и примеров, анализирует различные аспекты книги и дает развернутые рекомендации', 'review/helpfulness': '15/15'}
    ]
    
    print("1. MAP этап - обработка каждой строки:")
    mapped_results = []
    for i, row in enumerate(sample_data):
        results = list(do_map_length_helpfulness(row))
        mapped_results.extend(results)
        print(f"  Строка {i+1}: {results}")
    
    print("\n2. SHUFFLE этап - группировка по ключам:")
    shuffled = do_shuffle_length_helpfulness(iter(mapped_results))
    for key, values in shuffled.items():
        print(f"  Группа '{key}': {values}")
    
    print("\n3. REDUCE этап - агрегация данных:")
    reduced = do_reduce_length_helpfulness(shuffled)
    for key, value in reduced:
        print(f"  '{key}' -> средняя полезность: {value:.3f}")

In [30]:
# Загрузка данных
def load_data() -> pd.DataFrame:
    try:
        df_rating = pd.read_csv('Books_rating.csv')
        df_books = pd.read_csv('Books_data.csv')
        return pd.merge(df_rating, df_books[['Title', 'categories']], on='Title', how='left')
    except FileNotFoundError:
        print("Файлы данных не найдены. Создаем тестовый DataFrame.")
        return pd.DataFrame({
            'review/text': ['Тестовый короткий отзыв', 'Средний отзыв о книге', 'Длинный подробный анализ книги с множеством деталей'],
            'review/helpfulness': ['2/3', '5/5', '10/10'],
            'categories': ["['Fiction']", "['Science Fiction']", "['Literary Fiction, Classic']"]
        })

# Основной анализ
df = load_data().head(10000)

In [33]:
# Демонстрация работы Map-Reduce
demonstrate_map_reduce()


=== ДЕМОНСТРАЦИЯ РАБОТЫ MAP-REDUCE ===
1. MAP этап - обработка каждой строки:
  Строка 1: [('short', 0.6)]
  Строка 2: [('short', 0.8)]
  Строка 3: [('short', 1.0)]

2. SHUFFLE этап - группировка по ключам:
  Группа 'short': [0.6, 0.8, 1.0]

3. REDUCE этап - агрегация данных:
  'short' -> средняя полезность: 0.800


In [37]:
# Анализ влияния длины на полезность
print("\n\n=== АНАЛИЗ ВЛИЯНИЯ ДЛИНЫ РЕЦЕНЗИИ НА ПОЛЕЗНОСТЬ ===")
mapped_length_help = []
for _, row in df.iterrows():
    mapped_length_help.extend(do_map_length_helpfulness(row))

shuffled_length_help = do_shuffle_lenght_helpfulness(iter(mapped_length_help))
reduced_length_help = do_reduce_length_helpfulness(shuffled_length_help)

print("Результаты анализа:")
for cat, avg_ratio in sorted(reduced_length_help, key=lambda x: x[1], reverse=True):
    print(f"{cat:10}  {avg_ratio:.3f}")

# Анализ жанров с подробными обсуждениями
print("\n\n=== АНАЛИЗ ЖАНРОВ С НАИБОЛЕЕ ПОДРОБНЫМИ ОБСУЖДЕНИЯМИ ===")

mapped_genre_length = []
for _, row in df.iterrows():
    mapped_genre_length.extend(do_map_genre_length(row))

shuffled_genre_length = do_shuffle_genre_length(iter(mapped_genre_length))
reduced_genre_length = do_reduce_genre_length(shuffled_genre_length)

print("\nТоп-10 жанров по средней длине отзывов:")
for genre, avg_length in sorted(reduced_genre_length, key=lambda x: x[1], reverse=True)[:10]:
    print(f"{genre:40}  {avg_length:.1f} слов")



=== АНАЛИЗ ВЛИЯНИЯ ДЛИНЫ РЕЦЕНЗИИ НА ПОЛЕЗНОСТЬ ===
Результаты анализа:
long        0.739
medium      0.639
short       0.396


=== АНАЛИЗ ЖАНРОВ С НАИБОЛЕЕ ПОДРОБНЫМИ ОБСУЖДЕНИЯМИ ===

Топ-10 жанров по средней длине отзывов:
Authorship                                610.0 слов
Heroes                                    605.0 слов
Great Britain                             443.5 слов
Humanism                                  429.0 слов
Economics                                 403.0 слов
Design                                    351.0 слов
Ecoterrorism                              306.3 слов
Church and state                          289.6 слов
Mythology, Classical                      277.7 слов
Authors, English                          245.7 слов
